In [2]:
pip install selenium


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bs4


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
#IMPORTACIONES
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import csv
import requests
import pandas as pd
import re
import os

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [23]:
#WEBDRIVER
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Modo headless
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [ ]:
#HOJA DE DATOS DE VINOS

import os
import re
import csv
import time
import requests
from bs4 import BeautifulSoup

# Archivo de entrada
input_file = "/Users/josetudela/Projects/vinos_grupo2/Proyecto_grupo2_vinos/vinos_convers/tintos_1_15000.txt"

# Archivo de salida
output_file = "wine_data.csv"

# Archivo de punto de control (checkpoint)
checkpoint_file = "last_processed_batch.txt"

# Headers para evitar bloqueos
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Leer todas las URLs del archivo de texto
with open(input_file, 'r', encoding='utf-8-sig') as f:
    urls = [line.strip() for line in f.readlines() if line.strip().startswith("http")]

# Dividir la lista en lotes de 100
batch_size = 100
num_batches = (len(urls) // batch_size) + (1 if len(urls) % batch_size > 0 else 0)

# Verificar el último lote procesado desde el archivo de punto de control
last_processed_batch = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        last_processed_batch = int(f.read().strip())

print(f"🔹 Total de URLs: {len(urls)}")
print(f"🔹 Procesando en {num_batches} lotes de {batch_size} URLs cada uno.")
print(f"🔹 Comenzando desde el lote {last_processed_batch + 1}...")

# Procesar cada batch desde el último lote procesado
for batch_index in range(last_processed_batch, num_batches):
    batch_urls = urls[batch_index * batch_size: (batch_index + 1) * batch_size]
    all_wine_data = []
    
    print(f"\n🚀 Procesando lote {batch_index + 1} de {num_batches}...")

    for url in batch_urls:
        print(f"🔄 Procesando URL: {url}")  # Mostrar la URL que se está procesando
        
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Datos básicos
            wine_data = {}
            breadcrumbs = soup.find(class_='breadCrumbs__breadCrumbs--2pkcX')
            country = breadcrumbs.find(attrs={'data-cy': 'breadcrumb-country'}).text.strip() if breadcrumbs else 'No disponible'
            region = breadcrumbs.find(attrs={'data-cy': 'breadcrumb-region'}).text.strip() if breadcrumbs else 'No disponible'
            winery = breadcrumbs.find(attrs={'data-cy': 'breadcrumb-winery'}).text.strip() if breadcrumbs else 'No disponible'
            wine_type = breadcrumbs.find(attrs={'data-cy': 'breadcrumb-winetype'}).text.strip() if breadcrumbs else 'No disponible'

            # Uva
            grapes = soup.find_all("a", class_="anchor_anchor__m8Qi- wineFacts__link--3aTg9")
            grape = [g.text.strip() for g in grapes if "grapes" in g["href"]]
            grape = ', '.join(grape) if grape else 'No disponible'

            # Nombre del vino
            wine_headline = soup.find('div', class_='wineHeadline-module__wineHeadline--32Ety')
            name = wine_headline.get_text(strip=True) if wine_headline else 'No disponible'
            if winery.lower() in name.lower():
                name = name.replace(winery, '').strip()

            # Año
            year = 'No disponible'
            button_elements = soup.find_all('button', class_='MuiButtonBase-root')
            for button in button_elements:
                if button.get('aria-label') and button.get('aria-label').isdigit() and len(button.get('aria-label').strip()) == 4:
                    year = button.get('aria-label').strip()
                    break

            # Precio
            price_element = soup.find(class_='purchaseAvailability__currentPrice--3mO4u')
            if price_element:
                price = price_element.text.replace('€', '').replace('\xa0', '').strip()
            else:
                price_element = soup.find(class_='purchaseAvailabilityPPC__amount--2_4GT')
                match = re.search(r'\d{1,3}(?:,\d{3})*(?:\.\d+)?', price_element.text) if price_element else None
                price = match.group(0).replace('\xa0', '').strip() if match else 'No disponible'

            # Grados de Alcohol
            alcohol = 'No disponible'
            alcohol_element = soup.find(class_='wineFacts__wineFacts--2Ih8B')
            if alcohol_element:
                spans = alcohol_element.find_all('span')
                for span in spans:
                    match = re.search(r'\d+%', span.text.strip())
                    if match:
                        alcohol = match.group(0).replace('%', '')
                        break

            # Notas de sabor
            taste_containers = soup.find_all(class_='slider__viewPort--30MrB')
            taste_notes = []
            for container in taste_containers:
                taste_keywords = container.find_all(class_='tasteNote__popularKeywords--1gIa2')
                taste_notes.extend([keyword.text.strip() for keyword in taste_keywords if keyword.text.strip()])
            taste_notes = ', '.join(taste_notes) if taste_notes else 'No disponible'

            # Valoración
            rating_element = soup.find(class_='vivinoRating_averageValue__uDdPM')
            rating = rating_element.text.strip().replace(',', '.') if rating_element else 'No disponible'

            # Maridajes
            food_pairings = soup.find_all(class_='foodPairing__foodImage--2OYHg')
            pairings = [fp['aria-label'] for fp in food_pairings if fp.has_attr('aria-label')]

            # Guardar los datos
            wine_data = {
                'Url': url,
                'ID': re.search(r'\/(\d+)$', url).group(1) if re.search(r'\/(\d+)$', url) else '0',
                'Nombre': name,
                'Año': year,
                'País': country,
                'Región': region,
                'Bodega': winery,
                'Tipo de vino': wine_type,
                'Uva': grape,
                'Precio': price,
                'Valoración': rating,
                'Contenido de alcohol': alcohol,
                'Maridajes': ', '.join(pairings),
            }
            all_wine_data.append(wine_data)

        except Exception as e:
            print(f"❌ Error al procesar la URL {url}: {e}")

    # Guardar en un archivo CSV único para cada batch
    batch_output_file = f"wine_data_batch_{batch_index + 1}.csv"
    file_exists = os.path.exists(batch_output_file)

    with open(batch_output_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=all_wine_data[0].keys())
        if not file_exists:
            writer.writeheader()  # Escribir los encabezados solo la primera vez
        writer.writerows(all_wine_data)

    print(f"✅ Lote {batch_index + 1} guardado en {batch_output_file}")

    # Actualizar el archivo de punto de control con el índice del lote procesado
    with open(checkpoint_file, 'w') as f:
        f.write(str(batch_index))

    # Pequeña pausa entre lotes para evitar bloqueos
    time.sleep(5)

print("\n🎯 ¡Todos los lotes han sido procesados y guardados correctamente! 🚀")

🔹 Total de URLs: 15200
🔹 Procesando en 152 lotes de 100 URLs cada uno.
🔹 Comenzando desde el lote 43...

🚀 Procesando lote 43 de 152...
🔄 Procesando URL: https://www.vivino.com/ES/es/valentini-abruzzo-montepulciano-d-abruzzo/w/1105697
🔄 Procesando URL: https://www.vivino.com/ES/es/antinori-tuscany-tenuta-guado-al-tasso-matarocchio/w/1415698
🔄 Procesando URL: https://www.vivino.com/ES/es/giacomo-conterno-barolo-arione/w/7632000
🔄 Procesando URL: https://www.vivino.com/ES/es/quintarelli-giuseppe-veneto-alzero-cabernet/w/1194532
🔄 Procesando URL: https://www.vivino.com/ES/es/montevertine-le-pergole-torte/w/15292
🔄 Procesando URL: https://www.vivino.com/ES/es/oasi-degli-angeli-kurni-marche/w/1755765
🔄 Procesando URL: https://www.vivino.com/ES/es/castello-del-terriccio-gian-annibale/w/11165977
🔄 Procesando URL: https://www.vivino.com/ES/es/bruno-giacosa-barolo-barolo-red-wine/w/8717167
🔄 Procesando URL: https://www.vivino.com/ES/es/tenuta-del-nicchio-lodovico/w/1320744
🔄 Procesando URL: htt

In [1]:
#BARRA DE SABOR
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import re
import pandas as pd

# Configuración del WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Inicializar WebDriver
driver = webdriver.Chrome(options=options)

# Leer URLs desde el archivo
txt_file = '/Users/josetudela/Projects/vinos_grupo2/Proyecto_grupo2_vinos/vinos_convers/tintos_1_15000_limpio.txt'
with open(txt_file, 'r') as file:
    urls_list = [line.strip() for line in file.readlines()]

# Mapeo de etiquetas y clases de las barras de progreso
labels = ["Ligero/Poderoso", "Suave/Tánico", "Seco/Dulce", "Débil/Ácido"]

# Nombre del archivo CSV
csv_filename = "caracteristicas_tintos.csv"

# Cargar datos existentes o crear un nuevo DataFrame
if os.path.exists(csv_filename):
    df = pd.read_csv(csv_filename)
    processed_ids = set(df["ID"].astype(str))  # Convertir IDs a string para evitar problemas
    print("Ejemplo de IDs procesados:", list(processed_ids)[:10])
else:
    df = pd.DataFrame(columns=["ID"] + labels)
    processed_ids = set()

# Filtrar URLs que ya fueron procesadas
urls_list = [url for url in urls_list if re.search(r'\/(\d+)$', url) and re.search(r'\/(\d+)$', url).group(1) not in processed_ids]
print(f"Quedan {len(urls_list)} URLs por procesar.")

# Procesar en lotes de 100
batch_size = 100
for batch_start in range(0, len(urls_list), batch_size):
    batch_urls = urls_list[batch_start:batch_start + batch_size]
    print(f"Procesando lote {batch_start // batch_size + 1} ({len(batch_urls)} URLs)")
    
    for original_url in batch_urls:
        print(f"Procesando URL: {original_url}")
        match = re.search(r'\/(\d+)$', original_url)
        ID = match.group(1) if match else "Desconocido"

        try:
            driver.get(original_url)
            WebDriverWait(driver, 10).until(lambda d: len(d.find_elements(By.CLASS_NAME, 'indicatorBar__progress--3aXLX')) >= len(labels))

            progress_elements = driver.find_elements(By.CLASS_NAME, 'indicatorBar__progress--3aXLX')
            
            if len(progress_elements) == len(labels):
                progress_values = {labels[i]: round(float(re.search(r'left:\s*([0-9.]+)', el.get_attribute('style')).group(1)) / 10, 1) for i, el in enumerate(progress_elements)}
                progress_values["ID"] = ID

                if ID in df["ID"].values:
                    df.loc[df["ID"] == ID, labels] = [progress_values[label] for label in labels]
                else:
                    df = pd.concat([df, pd.DataFrame([progress_values])], ignore_index=True)
            else:
                print("Número de barras de progreso inesperado.")

        except Exception as e:
            print(f"Error con {original_url}: {e}")
    
    # Guardar el CSV al finalizar el lote
    df.to_csv(csv_filename, index=False)
    print(f"✅ Lote {batch_start // batch_size + 1} guardado en {csv_filename}")

# Cerrar el WebDriver
driver.quit()
print("🎉 Proceso completado.")



Ejemplo de IDs procesados: ['4630455', '1995070', '1574127', '10491085', '82392', '1207562', '98956', '5437585', '9844090', '9585401']
Quedan 324 URLs por procesar.
Procesando lote 1 (100 URLs)
Procesando URL: https://www.vivino.com/ES/es/brown-brothers-tarrango/w/23747
Error con https://www.vivino.com/ES/es/brown-brothers-tarrango/w/23747: Message: 

Procesando URL: https://www.vivino.com/ES/es/zagreus-tiara-mavrud/w/1438894
Error con https://www.vivino.com/ES/es/zagreus-tiara-mavrud/w/1438894: Message: 

Procesando URL: https://www.vivino.com/ES/es/san-michele-it-tridentum-teroldego/w/1287100
Error con https://www.vivino.com/ES/es/san-michele-it-tridentum-teroldego/w/1287100: Message: 

Procesando URL: https://www.vivino.com/ES/es/garage-wine-co-la-higuera-vineyard-cabernet-franc-lot/w/5328426
Error con https://www.vivino.com/ES/es/garage-wine-co-la-higuera-vineyard-cabernet-franc-lot/w/5328426: Message: 

Procesando URL: https://www.vivino.com/ES/es/haltinger-winzer-pinot-noir-blanc

In [7]:
#SABORIFY

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd  
import time
import os

# Configurar el WebDriver de Chrome con opciones avanzadas
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")  # Mejora estabilidad en algunos entornos
options.add_argument("--log-level=3")  # Reduce logs innecesarios
options.add_argument("--enable-logging")  # Activa logs para ver posibles fallos

driver = webdriver.Chrome(service=service, options=options)

# Leer las URLs desde el archivo
input_filename = "/Users/josetudela/Projects/vinos_grupo2/Proyecto_grupo2_vinos/Vinos blancos de 70 a 100.txt"
output_basename = os.path.splitext(os.path.basename(input_filename))[0]  # 

start_index = 1 # Cambia este valor para elegir desde qué fila comenzar

with open(input_filename, "r", encoding="utf-8") as file:
    urls = [url.strip() for url in file.readlines()]

urls = urls[901:]  # Cortar la lista desde la fila deseada


data = []
file_count = 10 # Contador de archivos
batch_size = 100  # Cada cuántos registros se guarda

# Función para hacer scroll
def scroll_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Esperar a que cargue más contenido

# Recorrer cada URL
for index, url in enumerate(urls, start=1):
    driver.get(url)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except Exception:
        print(f"⚠️ No se pudo cargar la página {url}, saltando...")
        continue

    scroll_page()

    # Extraer el ID del vino desde la URL
    wine_id = url.split("/")[-1]
    print(f"🔍 Procesando vino {index}/{len(urls)}: {wine_id}")

    try:
        # Esperar a que aparezcan las notas de sabor
        taste_notes_elements = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.tasteNote__popularKeywords--1gIa2"))
        )
        
        # Extraer y limpiar las notas de sabor
        taste_notes = [el.text.strip() for el in taste_notes_elements if el.text.strip()]
        print(f"✅ Notas de sabor para {wine_id}: {taste_notes}")

        # Estructurar los datos en columnas separadas
        taste_notes_dict = {f"Taste_Note_{i+1}": note for i, note in enumerate(taste_notes[:3])}

    except Exception:
        print(f"⚠️ No se encontraron notas de sabor para {wine_id}")
        taste_notes_dict = {"Taste_Note_1": "No disponible", "Taste_Note_2": "No disponible", "Taste_Note_3": "No disponible"}

    # Agregar datos a la lista
    data.append({"Wine_ID": wine_id, **taste_notes_dict})

    # Guardar cada `batch_size` registros
    if len(data) >= batch_size:
        output_filename = f"{output_basename}_{file_count}.csv"
        df = pd.DataFrame(data)
        df.to_csv(output_filename, index=False, encoding="utf-8-sig")
        print(f"💾 Guardado {batch_size} registros en {output_filename}")
        data.clear()  # Vaciar la lista para la siguiente tanda
        file_count += 1  # Incrementar el contador de archivos

# Guardar los registros restantes
if data:
    output_filename = f"{output_basename}_{file_count}.csv"
    df = pd.DataFrame(data)
    df.to_csv(output_filename, index=False, encoding="utf-8-sig")
    print(f"💾 Guardado {len(data)} registros finales en {output_filename}")

# Cerrar el driver
driver.quit()
print("✅ Proceso completado.")

🔍 Procesando vino 1/38: 6482395
✅ Notas de sabor para 6482395: ['manzana', 'miel', 'levadura']
🔍 Procesando vino 2/38: 5563270
✅ Notas de sabor para 5563270: ['roble, vainilla, manteq...', 'minerales, petróleo, pi...', 'albaricoque, melocotón,...']
🔍 Procesando vino 3/38: 2396179
✅ Notas de sabor para 2396179: ['minerales, miel, tiza', 'limón, lima, cítrico', 'pera, manzana, manzana ...']
🔍 Procesando vino 4/38: 12781836
⚠️ No se encontraron notas de sabor para 12781836
🔍 Procesando vino 5/38: 1690053
✅ Notas de sabor para 1690053: ['melocotón, manzana, per...', 'miel, minerales, piedra', 'paja, espárrago, hierba']
🔍 Procesando vino 6/38: 736426
✅ Notas de sabor para 736426: ['miel, piedra, panal de ...', 'albaricoque, drupa, mel...', 'naranja, mermelada, cít...']
🔍 Procesando vino 7/38: 1844702
✅ Notas de sabor para 1844702: ['manzana, melón, pera', 'miel, minerales, tiza', 'limón, cítrico, piel de...']
🔍 Procesando vino 8/38: 1545575
✅ Notas de sabor para 1545575: ['miel, minerales, 